In [13]:
#imports
import csv
import numpy as np
import string
import pandas as pd
import re
from string import digits

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy
en_core = spacy.load('en_core_web_sm')
nlp_lg = spacy.load('en_core_web_lg')
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [158]:
#binary
data = pd.read_csv("tweet_emotions.csv")
count = 0
for index, sent in enumerate(data.sentiment):
    if sent == "fun" or sent == "enthusiasm" or sent == "love" or sent == "happiness" or sent =="relief":
        data.loc[index,['sentiment']] = "positive"
    elif sent == "hate" or sent == "anger" or sent == "empty" or sent == "worry" or sent == "sadness" or sent == "boredom":
        data.loc[index,['sentiment']] = "negative"
    else:
        #if count % 6 != 0:
        data.drop([index], axis=0, inplace=True)

data.to_csv('tweet_emotions_binary.csv')

KeyboardInterrupt: 

In [108]:
data
data['sentiment'].value_counts()

negative    16063
positive    13112
Name: sentiment, dtype: int64

In [111]:
#three classes
data = pd.read_csv("tweet_emotions.csv")
count = 0
for index, sent in enumerate(data.sentiment):
    if sent == "fun" or sent == "enthusiasm" or sent == "love" or sent == "happiness" or sent =="relief":
        data.loc[index,['sentiment']] = "positive"
    elif sent == "hate" or sent == "anger" or sent == "empty" or sent == "worry" or sent == "sadness" or sent == "boredom":
        data.loc[index,['sentiment']] = "negative"
    elif sent == "surprise":
        data.drop([index], axis=0, inplace=True)
data.to_csv('tweet_emotions_three_classes.csv')

In [112]:
data
data['sentiment'].value_counts()





negative    16063
positive    13112
neutral      8638
Name: sentiment, dtype: int64

Data cleaning functions

In [18]:

def remove_punctuation(line_text):
    line_text_without_punct = re.sub(r'[^\w\s]', '', line_text)
    return line_text_without_punct


def remove_numbers(line_text):
    table = str.maketrans('', '', digits)
    line_text_without_nums = line_text.translate(table)
    return line_text_without_nums

def remove_url(line_text):
    #(https?:\/\/) matches http:// or https://
    #(\s)* optional whitespaces
    #(www\.)? optionally matches www.
    # (\s)* optionally matches whitespaces
    #'((\w|\s)+\.)* matches 0 or more of one or more word characters followed by a period
    #([\w\-\s]+\/)* matches 0 or more of one or more words(or a dash or a space) followed by '\'
    #([\w\-]+) any remaining path at the end of the url followed by an optional ending
    #((\?)?[\w\s]*=\s*[\w\%&]*)* matches ending query params (even with white spaces,etc)

    line_text_without_url = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', line_text)
    return line_text_without_url

def remove_username(line_text):
    line_text_without_username = re.sub("([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)",'', line_text)
    return line_text_without_username

def remove_stopwords(line_text):
    line_text_without_stopwords = ' '.join([word for word in line_text.split()
                                           if word not in (stopwords.words('english'))])
    return line_text_without_stopwords

def translate_abbreviations(line_text):
    text_list = line_text.split()
    with open("slang_abbreviations.txt", 'r') as my_csv_file:
        data_from_file = csv.reader(my_csv_file, delimiter="=")
        for index, _str in enumerate(text_list):
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in data_from_file:
                if _str.upper() == row[0]:
                    text_list[index] = row[1]
    final_string = ' '.join(text_list)
    return final_string

def remove_hashtags(line_text):
    line_text_without_hashtags = line_text.replace('#','')
    return line_text_without_hashtags

def lemmatizing(line_text):
    final_string = " ".join([word.lemma_ for word in en_core(line_text)])
    return final_string

def clean_line(line_text):
    text_zero = remove_numbers(line_text)
    text_one = remove_username(text_zero)
    text_two = remove_url(text_one)
    text_three = remove_punctuation(text_two)
    text_four = remove_hashtags(text_three)
    text_five = remove_stopwords(text_four)
    text_six = translate_abbreviations(text_five)
    text_final = lemmatizing(text_six)
    return text_final



Cleaning the text and saving cleaned version to new column 'cleaned_content'

In [19]:
df_multiple = pd.read_csv("tweet_emotions.csv")
df_binary = pd.read_csv("tweet_emotions_binary.csv")
df_three_classes = pd.read_csv("tweet_emotions_three_classes.csv")

In [20]:
df_multiple['cleaned_content'] = df_multiple.content.apply(clean_line)
df_binary['cleaned_content'] = df_binary.content.apply(clean_line)
df_three_classes['cleaned_content'] = df_three_classes.content.apply(clean_line)

Saving cleaned Dataframe to scv for future

In [21]:
df_binary.to_csv('tweet_emotions_binary_cleaned.csv')
df_three_classes.to_csv('tweet_emotions_three_classes_cleaned.csv')
df_multiple.to_csv('tweet_emotions_cleaned.csv')

In [165]:
df_binary

,Unnamed: 0,tweet_id,sentiment,content,cleaned_content
0,0,1956967341,negative,@tiffanylue i know i was listenin to bad habi...,know listenin bad habit early start freakin part
1,1,1956967666,negative,Layin n bed with a headache ughhhh...waitin o...,Layin n bed headache ughhhhwaitin call
2,2,1956967696,negative,Funeral ceremony...gloomy friday...,funeral ceremonygloomy friday
3,3,1956967789,positive,wants to hang out with friends SOON!,want hang friend soon
4,5,1956968477,negative,Re-pinging @ghostridah14: why didn't you go to...,repinging do not go prom BC bf do not like friend
...,...,...,...,...,...
29170,39994,1753918900,positive,Succesfully following Tayla!!,succesfully follow tayla
29171,39996,1753919001,positive,Happy Mothers Day All my love,happy Mothers Day all love
29172,39997,1753919005,positive,Happy Mother's Day to all the mommies out ther...,happy Mothers Day mommie woman man long you re...
29173,39998,1753919043,positive,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,wassup BEAUTIFUL FOLLOW I peep out my new HIT ...


function which vectorize text and give the train and test data back

In [91]:
def SpacyVec_data_from_df(df_input):
    df = df_input
    df['text_tokenized'] = df.clened_content.apply(lambda x: nlp_lg(x)) # tokenizing every text in the dataframe

    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    #doing sentence embedding before splitting, because the order makes no difference
    df['text_vectorized'] = df['text_tokenized'].apply(lambda x: x.vector)

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.text_vectorized, df.Numeric_Labels,
                                                                 test_size=0.30,
                                                                 random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

    # appending all array rows into one and reshaping them from (1,300) shape to (300,1) shape
    data_list_train = [vec.reshape(1,-1) for vec in data_train]
    data_list_test = [vec.reshape(1,-1) for vec in data_test]

    # concentrating array to get them readable by sklearn models
    data_train_reshaped = np.concatenate(data_list_train)
    data_test_reshaped = np.concatenate(data_list_test)

    return data_train_reshaped, data_test_reshaped, label_train, label_test





In [2]:
def TfIdf_data_from_df(df_input):
    df = df_input
    df.sentiment = pd.Categorical(df.sentiment) # changing type of a column to categorical
    df['Numeric_Labels'] = df.sentiment.cat.codes # saving category codes of column labels to new column

    # splitting data _________________COULD BE CHANGED____________________________________________________
    data_train, data_test, label_train, label_test = train_test_split(df.cleaned_content, df.Numeric_Labels, test_size=0.30, random_state=101, shuffle=True)
   # __________________________________________________________________________________________________________

   # vectorizing test and train data with Tfidf
    polarity_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    polarity_bow_matrix_train = polarity_tfidf_vectorizer.fit_transform(data_train)
    polarity_bow_matrix_test = polarity_tfidf_vectorizer.transform(data_test)


    return polarity_bow_matrix_train, polarity_bow_matrix_test, label_train, label_test


In [7]:
df_binary = pd.read_csv('tweet_emotions_binary_cleaned.csv')
data_train, data_test, label_train, label_test = TfIdf_data_from_df(df_binary)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [97]:
forest_classifier = RandomForestClassifier()

In [85]:
forest_classifier = RandomForestClassifier()
forest_classifier.fit(data_train, label_train)
test_forest= forest_classifier.predict(data_test)

#evaluationg model and reporting accuracy and multi-class confusion matrix
print('\nEvaluate MLPClassifier:')
print('\nAccuracy: %.3f' % metrics.accuracy_score(label_test, test_forest))
print('\nConfusion Matrix: ')
multilabel_confusion_matrix(label_test, test_forest)


Evaluate MLPClassifier:

Accuracy: 0.330

Confusion Matrix: 


array([[[11966,     0],
        [   34,     0]],

       [[11946,     4],
        [   50,     0]],

       [[11704,    33],
        [  261,     2]],

       [[11775,    13],
        [  212,     0]],

       [[11390,    81],
        [  513,    16]],

       [[ 9408,  1001],
        [ 1109,   482]],

       [[11519,    86],
        [  341,    54]],

       [[10323,   589],
        [  684,   404]],

       [[ 6056,  3322],
        [ 1055,  1567]],

       [[11442,    69],
        [  471,    18]],

       [[ 9867,   517],
        [ 1292,   324]],

       [[11250,   122],
        [  600,    28]],

       [[ 7316,  2201],
        [ 1416,  1067]]])

In [ ]:
def train_models(df, vectorizer, text="text", labels="emotion"):
    classifiers = [svm.LinearSVC(), RandomForestClassifier()] #DecisionTreeClassifier()]# # MLPClassifier()]

    #look into learning curves (do we have enough samples?) / validation curves (is the model under/overfitting?)

    X = vect.transform(df[text])
    y = df[labels]

    print("Amount of features: ", len(vect.get_feature_names_out()))
    print("Features: ", vect.get_feature_names_out())

    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    plot_learning_curve(classifiers, X, y, ylim=(0.5, 1.01), cv=cv, n_jobs=4)

    results = calc_metrics(classifiers, X, y)

    df_results = pd.DataFrame(results)
    return df_results

    plt.show(